In [4]:
import sys
sys.executable

'C:\\Users\\vylo0\\analisis_datos_env\\Scripts\\python.exe'

In [5]:
import pandas as pd
import sqlite3
from datetime import datetime

pd.set_option("display.max_columns", None)

## 1. Identificación de fuentes de datos

El pipeline ETL integra datos provenientes de múltiples sistemas empresariales,
cada uno con características y formatos distintos:

- **Sistema de Punto de Venta (POS)**  
  - Tipo: Base de datos SQL  
  - Datos: Ventas por tienda, producto y hora  

- **Sistema de Inventario**  
  - Tipo: API REST  
  - Datos: Stock por producto y ubicación  

- **CRM**  
  - Tipo: Archivos CSV exportados  
  - Datos: Información de clientes y segmentación  

- **Sitio Web**  
  - Tipo: Registros del servidor en formato JSON  
  - Datos: Comportamiento online de los clientes  

Estas fuentes representan silos de datos que deben integrarse para un análisis unificado.

## 2. Problema de datos fragmentados

En la organización, los datos se encuentran distribuidos en distintos sistemas
que no están integrados entre sí, lo que genera múltiples problemas:

- Información de ventas separada del inventario y del CRM
- Diferentes formatos de fecha y hora entre sistemas
- Códigos de producto inconsistentes
- Canales de venta online y tienda física aislados
- Dificultad para cruzar información de clientes con ventas

Esta fragmentación provoca retrasos en el análisis y dificulta la toma de decisiones
basadas en una visión completa del negocio.


## 3. Esquema destino: Tabla unificada de ventas

El sistema destino del proceso ETL es una base de datos analítica que consolida
la información de ventas en una única tabla de hechos.

```sql
CREATE TABLE ventas_consolidadas (
    id_venta INTEGER PRIMARY KEY,
    fecha_venta DATE,
    id_tienda INTEGER,
    id_cliente INTEGER,
    id_producto INTEGER,
    cantidad INTEGER,
    precio_unitario DECIMAL(10,2),
    total_venta DECIMAL(10,2),
    canal_venta VARCHAR(20),
    segmento_cliente VARCHAR(20)
);

In [6]:
import pandas as pd

# =========================
# POS - Sistema de ventas
# =========================
ventas_pos = pd.DataFrame({
    "id_venta": [1, 2, 3, 3],  # id 3 duplicado intencional
    "timestamp_venta": [
        "2025-12-01 10:15:00",
        "2025-12-01 11:30:00",
        "2025-12-01 12:45:00",
        "2025-12-01 12:45:00"
    ],
    "id_tienda": [101, 101, 102, 102],
    "id_cliente": [10, 20, 10, 10],
    "id_producto": ["P001", "P002", "P001", "P001"],
    "cantidad": [2, 1, 3, 3],
    "precio_unitario": [5000, 12000, 5000, 5000],
    "canal_venta": ["tienda", "tienda", "tienda", "tienda"]
})

# =========================
# CRM - Clientes
# =========================
clientes_crm = pd.DataFrame({
    "id_cliente": [10, 20, 30],
    "segmento_cliente": ["Premium", "Regular", "Nuevo"]
})

# =========================
# (Simulación) Web - Online
# =========================
web_logs = pd.DataFrame({
    "id_cliente": [10, 30],
    "timestamp_web": [
        "2025-12-01T09:50:00",
        "2025-12-01T12:10:00"
    ],
    "canal_venta": ["online", "online"]
})

In [5]:
ventas_pos

,id_venta,timestamp_venta,id_tienda,id_cliente,id_producto,cantidad,precio_unitario,canal_venta
0,1,2025-12-01 10:15:00,101,10,P001,2,5000,tienda
1,2,2025-12-01 11:30:00,101,20,P002,1,12000,tienda
2,3,2025-12-01 12:45:00,102,10,P001,3,5000,tienda
3,3,2025-12-01 12:45:00,102,10,P001,3,5000,tienda


In [7]:
# -------------------------
# 1. Convertir timestamp a fecha
# -------------------------
ventas_pos["fecha_venta"] = pd.to_datetime(
    ventas_pos["timestamp_venta"]
).dt.date

# -------------------------
# 2. Calcular total de venta
# -------------------------
ventas_pos["total_venta"] = (
    ventas_pos["cantidad"] * ventas_pos["precio_unitario"]
)

# -------------------------
# 3. Eliminar ventas duplicadas
# -------------------------
ventas_pos = ventas_pos.drop_duplicates(subset=["id_venta"])

# -------------------------
# 4. Enriquecer con segmento de cliente (CRM)
# -------------------------
ventas_transformadas = ventas_pos.merge(
    clientes_crm,
    on="id_cliente",
    how="left"
)

# -------------------------
# 5. Seleccionar columnas finales
# -------------------------
ventas_transformadas = ventas_transformadas[
    [
        "id_venta",
        "fecha_venta",
        "id_tienda",
        "id_cliente",
        "id_producto",
        "cantidad",
        "precio_unitario",
        "total_venta",
        "canal_venta",
        "segmento_cliente"
    ]
]

ventas_transformadas

,id_venta,fecha_venta,id_tienda,id_cliente,id_producto,cantidad,precio_unitario,total_venta,canal_venta,segmento_cliente
0,1,2025-12-01,101,10,P001,2,5000,10000,tienda,Premium
1,2,2025-12-01,101,20,P002,1,12000,12000,tienda,Regular
2,3,2025-12-01,102,10,P001,3,5000,15000,tienda,Premium


## 4. Frecuencia y estrategia de carga

El pipeline ETL se diseña considerando la naturaleza y volumen de cada fuente
de datos:

- **Ventas del POS**  
  - Estrategia: Carga incremental  
  - Frecuencia: Cada hora  
  - Justificación: Alta generación de datos y necesidad de información actualizada.

- **Sistema de inventario**  
  - Estrategia: Carga completa  
  - Frecuencia: Diaria  
  - Justificación: Volumen manejable y cambios menos frecuentes.

- **CRM**  
  - Estrategia: Carga incremental  
  - Frecuencia: Semanal  
  - Justificación: Datos maestros con baja variabilidad.

- **Analítica web**  
  - Estrategia: Carga por lotes  
  - Frecuencia: Diaria  
  - Justificación: Procesamiento eficiente de logs de navegación.

Este enfoque equilibra frescura de datos, rendimiento y costos operativos.

## 5. Verificación y resolución del problema de datos fragmentados

El diseño del pipeline ETL permite resolver los principales problemas de
fragmentación de datos identificados inicialmente:

- Integra múltiples sistemas en una única tabla de hechos
- Estandariza formatos de fecha y métricas de ventas
- Enriquce las ventas con información de clientes
- Elimina duplicados e inconsistencias
- Proporciona una única fuente de verdad para el análisis

Como resultado, la organización obtiene una visión unificada del negocio,
facilitando análisis rápidos, confiables y orientados a la toma de decisiones.

In [8]:
import sqlite3

# Crear (o conectar a) base de datos SQLite
conexion = sqlite3.connect("etl_ventas.db")

In [9]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS ventas_consolidadas (
    id_venta INTEGER PRIMARY KEY,
    fecha_venta DATE,
    id_tienda INTEGER,
    id_cliente INTEGER,
    id_producto TEXT,
    cantidad INTEGER,
    precio_unitario REAL,
    total_venta REAL,
    canal_venta TEXT,
    segmento_cliente TEXT
);
"""

cursor = conexion.cursor()
cursor.execute(create_table_sql)
conexion.commit()

In [10]:
ventas_transformadas.to_sql(
    "ventas_consolidadas",
    conexion,
    if_exists="replace",  # para el ejercicio
    index=False
)

3

In [11]:
consulta = "SELECT * FROM ventas_consolidadas;"
ventas_sqlite = pd.read_sql_query(consulta, conexion)

ventas_sqlite

,id_venta,fecha_venta,id_tienda,id_cliente,id_producto,cantidad,precio_unitario,total_venta,canal_venta,segmento_cliente
0,1,2025-12-01,101,10,P001,2,5000,10000,tienda,Premium
1,2,2025-12-01,101,20,P002,1,12000,12000,tienda,Regular
2,3,2025-12-01,102,10,P001,3,5000,15000,tienda,Premium


In [12]:
import pandas as pd
import sqlite3

# Conectar a SQLite
conexion = sqlite3.connect("ventas_etl.db")

# Leer tabla final
df_excel = pd.read_sql_query(
    "SELECT * FROM ventas_consolidadas",
    conexion
)

# Exportar a Excel
df_excel.to_excel(
    "ventas_consolidadas.xlsx",
    index=False
)

conexion.close()

DatabaseError: Execution failed on sql 'SELECT * FROM ventas_consolidadas': no such table: ventas_consolidadas

In [13]:
import sqlite3
import pandas as pd

conexion = sqlite3.connect("ventas_etl.db")

pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conexion
)

,name


In [14]:
ventas_transformadas.to_sql(
    "ventas_consolidadas",
    conexion,
    if_exists="replace",
    index=False
)

3

In [15]:
pd.read_sql_query(
    "SELECT * FROM ventas_consolidadas",
    conexion
)

,id_venta,fecha_venta,id_tienda,id_cliente,id_producto,cantidad,precio_unitario,total_venta,canal_venta,segmento_cliente
0,1,2025-12-01,101,10,P001,2,5000,10000,tienda,Premium
1,2,2025-12-01,101,20,P002,1,12000,12000,tienda,Regular
2,3,2025-12-01,102,10,P001,3,5000,15000,tienda,Premium


In [16]:
ventas_transformadas.to_excel(
    "ventas_consolidadas.xlsx",
    index=False
)